In [1]:
decode = ['''0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111'''][0].split('\n')

In [2]:
hexvals = [elem.split(' = ')[0] for elem in decode]
bitvals = [elem.split(' = ')[1] for elem in decode]

In [3]:
decode = { hexval : bitval for hexval, bitval in zip(hexvals, bitvals) }

In [4]:
import math

In [5]:
total_version = 0

def unpack(packet):
    global total_version
    version = int(packet[0:3], base = 2)
    typeID = int(packet[3:6], base = 2)
    total_version += version
    if typeID == 4:
        last = False
        number = ''
        index = 6
        while not last:
            if packet[index] == '0': 
                last = True
            number += packet[index+1:index+5]
            index += 5
        no = int(number, base = 2)
        return(packet[:index], no)
    else:
        length, nsp = 0, 0
        ID = packet[6]
        numberlist = []
        answer = 0
        if ID == '0':
            length = int(packet[7:22], base = 2)
            subpackets = packet[22:22+length]
            while '1' in subpackets:
                ind, val = unpack(subpackets)
                subpackets = subpackets[len(ind):]
                numberlist.append(val)
            
            match typeID:
                case 0:
                    answer = sum(numberlist)
                case 1:
                    answer = math.prod(numberlist)
                case 2:
                    answer = min(numberlist)
                case 3:
                    answer = max(numberlist)
                case 5:
                    if numberlist[0] > numberlist[1]: answer = 1
                    else: answer = 0
                case 6:
                    if numberlist[0] < numberlist[1]: answer = 1
                    else: answer = 0
                case 7:
                    if numberlist[0] == numberlist[1]: answer = 1
                    else: answer = 0
                    
            return(packet[:22+length], answer)
        elif ID == '1':
            nsp = int(packet[7:18], base = 2)
            subpackets = packet[18:]
            howfar = 18
            for i in range(0, nsp):
                ind, val = unpack(subpackets)
                howfar += len(ind)
                numberlist.append(val)
                subpackets = subpackets[len(ind):]
                
            match typeID:
                case 0:
                    answer = sum(numberlist)
                case 1:
                    answer = math.prod(numberlist)
                case 2:
                    answer = min(numberlist)
                case 3:
                    answer = max(numberlist)
                case 5:
                    if numberlist[0] > numberlist[1]: answer = 1
                    else: answer = 0
                case 6:
                    if numberlist[0] < numberlist[1]: answer = 1
                    else: answer = 0
                case 7:
                    if numberlist[0] == numberlist[1]: answer = 1
                    else: answer = 0
                    
            return(packet[:howfar], answer)
        

In [6]:
infile = open('input.txt')
inpacket = [elem.strip() for elem in infile.readlines()][0]

code = ''
for char in inpacket:
    code += decode[char]               

p, ans = unpack(code)

print(total_version) #part1
print(ans) #part2

963
1549026292886
